In [97]:
import numpy as np
import matplotlib.pyplot as plt
import time
import random
from openai import OpenAI
import cProfile
from GNBG_instances import get_gnbg
from deepseek import get_rmp_update_code

In [92]:
np.random.seed(0)
np.set_printoptions(legacy='1.25')

In [93]:
class MFEA:
    def __init__(self, pop_size, indi_len, tasks):
        self.pop_size = pop_size
        self.indi_len = indi_len
        self.pop = np.random.uniform(-100.0, 100.0, size=(self.pop_size, self.indi_len))
        self.tasks = tasks
        self.num_tasks = len(tasks)
        self.rmp_update_code = ""
        self.skill_factor = np.zeros(self.pop_size, dtype=int)
    
        for i in range(self.pop_size):
            self.skill_factor[i] = i % self.num_tasks
        
        self.fitness = np.zeros(pop_size)
        self.mean_fitness = np.zeros(self.num_tasks)
        self.best_fitness = np.zeros(self.num_tasks)
        
        for task_id in range(self.num_tasks):
            task_mask = self.skill_factor == task_id
            self.fitness[task_mask] = self.tasks[task_id].fitness(self.pop[task_mask])
            self.mean_fitness[task_id] = np.mean(self.fitness[task_mask])
            self.best_fitness[task_id] = np.min(self.fitness[task_mask])

            print("Task {0}:".format(task_id))
            print("Mean Fitness At Initiation: {0}".format(self.mean_fitness[task_id]))
            print("Best Fitness At Initiation: {0}".format(self.best_fitness[task_id]))
            print("--------------------------------")
        self.cur_rmp = 0
        self.success_rate = 0

    def get_rmp_llm(self, get_new_code=False):
        if get_new_code == True:
            while True:
                try:
                    code = get_rmp_update_code()
                    print(code)
                    f = {}
                    exec(code, f)
                    rmp = f['get_rmp'](self.pop, self.fitness, self.skill_factor)
                    break
                except Exception as e:
                    print("Bad Function Generated!")
            self.rmp_update_code = code
            return rmp
        else:
            code = self.rmp_update_code
            f = {}
            exec(code, f)
            rmp = f['get_rmp'](self.pop, self.fitness, self.skill_factor)
        
        return rmp
    
    def get_rmp(self):
        return np.random.choice([0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], size=(self.pop_size, self.pop_size))

    def crossover(self, p1, p2):
        off = p1.copy()
        num_swap_pos = int(self.indi_len / 6)

        t = np.arange(len(off), dtype=np.int32) # t = [0, 1, 2, ... (len(off) - 1)]
        row_indices = np.lib.stride_tricks.as_strided(t, shape=(len(off), num_swap_pos), strides=(4, 0))
        
        swap_pos = np.random.randint(0, self.indi_len, size=(len(off), num_swap_pos))
        
        off[row_indices, swap_pos] = p2[row_indices, swap_pos]
        return off

    def mutate(self, p):
        off = p.copy()
        delta = np.random.uniform(-2, 2, size=off.shape)
        off += delta
        off[np.where(off > 100.0)] = 100.0
        off[np.where(off < -100.0)] = -100.0
        return off
    
    def run(self, num_gen, update_rmp=False, rf=0, llm=False, llm_rate=50):
        num_off = int(self.pop_size / 5)

        for gen in range(num_gen + 1):
            start_time = time.time()
            p1_indices = np.random.randint(0, self.pop_size, size=num_off)
            p2_indices = np.random.randint(0, self.pop_size, size=num_off)

            p1 = self.pop[p1_indices]
            p2 = self.pop[p2_indices]
            
            p1_skill_factor = self.skill_factor[p1_indices]
            p2_skill_factor = self.skill_factor[p2_indices]

            if llm == False:
                rmp = self.get_rmp()
            if llm == True:
                if gen % llm_rate == 0:
                    rmp = self.get_rmp_llm(get_new_code=True)
                else:
                    rmp = self.get_rmp_llm(get_new_code=False)
            self.cur_rmp = rmp
            
            intra_crossover_mask = np.equal(p1_skill_factor, p2_skill_factor)
            inter_crossover_mask = np.not_equal(p1_skill_factor, p2_skill_factor)
            
            intra_off = self.crossover(p1[intra_crossover_mask], p2[intra_crossover_mask]) #intra-mating
            intra_skill_factor = p1_skill_factor[intra_crossover_mask]

            rmp_values = rmp[p1_indices, p2_indices]
            
            rnd = np.random.uniform(size=len(p1_indices))
            
            rmp_accept_mask = np.greater(rmp_values, rnd) & inter_crossover_mask
            
            p1_inter_indices = p1_indices[rmp_accept_mask]
            p2_inter_indices = p2_indices[rmp_accept_mask]

            inter_off = self.crossover(self.pop[p1_inter_indices], self.pop[p2_inter_indices]) #inter-mating
            inter_skill_factor = self.skill_factor[p1_inter_indices]

            mutate_mask = np.invert(np.greater(rmp_values, rnd)) & inter_crossover_mask
            mutate_off_1 = self.mutate(p1[mutate_mask])
            mutate_off_2 = self.mutate(p2[mutate_mask])
            mutate_skill_factor_1 = p1_skill_factor[mutate_mask]
            mutate_skill_factor_2 = p2_skill_factor[mutate_mask]
            mutate_off = np.concatenate([mutate_off_1, mutate_off_2])
            mutate_skill_factor = np.concatenate([mutate_skill_factor_1, mutate_skill_factor_2])

            intermediate_pop = np.concatenate([self.pop, intra_off, inter_off, mutate_off])
            intermediate_skill_factor = np.concatenate([self.skill_factor, intra_skill_factor, inter_skill_factor, mutate_skill_factor])

            self.pop = np.array([]).reshape(0, self.indi_len)
            self.fitness = []
            self.skill_factor = []

            for task_id in range(self.num_tasks):
                survive_size = int(self.pop_size / self.num_tasks)
                task_mask = intermediate_skill_factor == task_id
                tpop = intermediate_pop[task_mask] #pop with specific task
                tpop_fitness = self.tasks[task_id].fitness(tpop)

                self.best_fitness[task_id] = np.min(tpop_fitness)
                survive_indices = np.argpartition(tpop_fitness, survive_size - 1)[:survive_size]
                self.mean_fitness[task_id] = np.mean(tpop_fitness[survive_indices])

                self.pop = np.concatenate([self.pop, tpop[survive_indices]])
                self.fitness = np.concatenate([self.fitness, tpop_fitness[survive_indices]])
                self.skill_factor = np.concatenate([self.skill_factor, np.full(survive_size, task_id)])
            assert(len(self.pop) == self.pop_size)
            if gen % 20 == 0:
                print("Gen {0}:".format(gen))
                for task_id in range(self.num_tasks):
                    print("Task {0}:".format(task_id))
                    print("    Best Fitness: {0}".format(self.best_fitness[task_id]))
                    print("    Mean Fitness: {0}".format(self.mean_fitness[task_id]))
                print(psutil.Process().memory_info().rss / (1024 * 1024))
                print("--- %.2f seconds ---" % (time.time() - start_time))                

In [94]:
def get_tasks(task_ids):
    tasks = []
    for task_id in task_ids:
        tasks.append(get_gnbg(task_id))
    return tasks

In [98]:
tasks = get_tasks([1, 2, 9, 10, 17, 18])

In [99]:
pop_size = len(tasks) * 200
mfea = MFEA(pop_size, 30, tasks)

Task 0:
Mean Fitness At Initiation: 169958.07923376016
Best Fitness At Initiation: 85062.7776468086
--------------------------------
Task 1:
Mean Fitness At Initiation: -701.3087036303684
Best Fitness At Initiation: -701.3662124426851
--------------------------------
Task 2:
Mean Fitness At Initiation: 1697336.6918461781
Best Fitness At Initiation: 126127.42688156506
--------------------------------
Task 3:
Mean Fitness At Initiation: 214562.79304379114
Best Fitness At Initiation: 73119.64940517231
--------------------------------
Task 4:
Mean Fitness At Initiation: 6858154.221528683
Best Fitness At Initiation: 3722293.7724119285
--------------------------------
Task 5:
Mean Fitness At Initiation: 156139.79262626817
Best Fitness At Initiation: 57369.79069691928
--------------------------------


In [90]:
mfea.run(500, llm=True)

import numpy as np

def get_rmp(pop, fitness, skill_factor):
    n = len(pop)
    task_indices = [np.where(skill_factor == i)[0] for i in np.unique(skill_factor)]
    num_tasks = len(task_indices)
    
    rmp = np.zeros((n, n))
    
    for i in range(num_tasks):
        for j in range(i, num_tasks):
            idx_i = task_indices[i]
            idx_j = task_indices[j]
            
            if i == j:
                rmp[np.ix_(idx_i, idx_j)] = 1.0
            else:
                pop_i = pop[idx_i]
                pop_j = pop[idx_j]
                
                distances = np.sqrt(np.sum((pop_i[:, np.newaxis] - pop_j) ** 2, axis=2))
                max_dist = np.max(distances)
                normalized_dist = distances / max_dist
                
                fitness_i = fitness[idx_i]
                fitness_j = fitness[idx_j]
                fit_diff = np.abs(fitness_i[:, np.newaxis] - fitness_j)
                max_fit_diff = np.max(fit_diff)
                normaliz

In [100]:
#test
for task_id in range(len(tasks)):
    print("Task {0} optimal value : {1}".format(task_id, tasks[task_id].OptimumValue))

Task 0 optimal value : -1081.9837994003399
Task 1 optimal value : -703.1328146165181
Task 2 optimal value : -884.7360096017693
Task 3 optimal value : -604.9748272222274
Task 4 optimal value : -5000
Task 5 optimal value : -5000
